# 🏋️‍♂️ Módulo 7: Treinamento e Pré-treinamento - A Academia dos LLMs!

**Por Pedro Nunes Guth** 🚀

---

Fala, pessoal! Chegamos no módulo mais suado do nosso curso! 💪

Tá, mas o que é treinamento e pré-treinamento? Imagina que você vai aprender a jogar futebol. Primeiro você precisa aprender o básico: chutar, correr, dominar a bola (isso é o **pré-treinamento**). Depois, você treina para jogar numa posição específica: atacante, zagueiro, goleiro (isso é o **fine-tuning**).

Com LLMs é a mesma coisa! Primeiro eles aprendem a "entender" linguagem em geral, depois são especializados para tarefas específicas.

**Bora entender como funciona essa academia dos modelos!** 🎯

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-07_img_01.png)

In [ ]:
# Setup inicial - Importando as bibliotecas que vamos usar
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import seaborn as sns
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')

# Configurações para os gráficos ficarem liiindos!
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔥 Setup pronto! Bora treinar alguns modelos!")
print(f"PyTorch version: {torch.__version__}")
print(f"Numpy version: {np.__version__}")

## 🎯 O que é Pré-treinamento?

Lembra dos **embeddings** e **tokens** que vimos nos módulos anteriores? Agora vamos ver como o modelo aprende a criar essas representações!

O **pré-treinamento** é como ensinar uma criança a ler. Você não começa ensinando Shakespeare, né? Começa com "O gato subiu no telhado". 

### As 3 Fases do Pré-treinamento:

1. **Coleta de Dados**: Pegamos MUUUITO texto da internet (livros, artigos, Wikipedia...)
2. **Tokenização**: Transformamos texto em números (já vimos isso no Módulo 4!)
3. **Treinamento Auto-supervisionado**: O modelo aprende a prever a próxima palavra

**Dica do Pedro**: O pré-treinamento é caro pra caramba! O GPT-3 custou uns 4.6 milhões de dólares para treinar. Por isso que a galera usa modelos já pré-treinados! 💰

In [ ]:
# Vamos simular um dataset de pré-treinamento simples
# Imagina que estamos treinando um modelo para entender português

# Dados de exemplo (bem simplificado!)
textos_pretreinamento = [
    "O gato subiu no telhado",
    "O cachorro late muito alto", 
    "A chuva molha a rua",
    "O sol brilha no céu",
    "As flores crescem no jardim",
    "O pássaro voa livre",
    "A lua ilumina a noite",
    "O vento balança as árvores"
]

# Vamos criar um vocabulário simples
todas_palavras = []
for texto in textos_pretreinamento:
    palavras = texto.lower().split()
    todas_palavras.extend(palavras)

# Criando vocabulário único
vocabulario = list(set(todas_palavras))
vocab_size = len(vocabulario)

# Mapeamento palavra -> índice
palavra_para_id = {palavra: i for i, palavra in enumerate(vocabulario)}
id_para_palavra = {i: palavra for palavra, i in palavra_para_id.items()}

print(f"📚 Vocabulário criado com {vocab_size} palavras únicas")
print(f"Primeiras 10 palavras: {vocabulario[:10]}")
print(f"\n🔢 Exemplo de tokenização:")
print(f"'{textos_pretreinamento[0]}' -> {[palavra_para_id[palavra] for palavra in textos_pretreinamento[0].lower().split()]}")

## 🧠 Como Funciona o Treinamento Auto-supervisionado?

Tá, mas como o modelo aprende sem ninguém dizer o que tá certo ou errado?

É aí que entra a **mágica**: o modelo tenta adivinhar a próxima palavra! É como completar a frase:

- "O gato subiu no ___" (resposta: telhado)
- "Hoje está fazendo muito ___" (resposta: calor/frio/sol)

### O Processo:
1. **Input**: "O gato subiu no"
2. **Predição**: Modelo chuta "telhado" (70% confiança)
3. **Target**: A palavra real é "telhado"
4. **Erro**: Se acertou, erro baixo. Se errou, erro alto
5. **Backpropagation**: Ajusta os pesos para acertar na próxima

Esse processo se repete trilhões de vezes! 🔄

In [ ]:
# Vamos criar um modelo simples para demonstrar o conceito
class ModeloSimples(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super().__init__()
        # Camada de embedding (lembra do Módulo 5?)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM para capturar sequências
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        
        # Camada final para prever a próxima palavra
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x):
        # x: [batch_size, sequence_length]
        embedded = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        lstm_out, _ = self.lstm(embedded)  # [batch_size, seq_len, hidden_dim]
        output = self.fc(lstm_out)  # [batch_size, seq_len, vocab_size]
        return output

# Criando o modelo
modelo = ModeloSimples(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo.parameters(), lr=0.001)

print(f"🤖 Modelo criado!")
print(f"Parâmetros: {sum(p.numel() for p in modelo.parameters())}")
print(f"\n📐 Arquitetura:")
print(modelo)

In [ ]:
# Vamos preparar os dados para treinamento
def preparar_dados(textos, palavra_para_id, seq_length=3):
    X, y = [], []
    
    for texto in textos:
        palavras = texto.lower().split()
        ids = [palavra_para_id[palavra] for palavra in palavras]
        
        # Criando sequências de entrada e saída
        for i in range(len(ids) - seq_length):
            X.append(ids[i:i+seq_length])
            y.append(ids[i+seq_length])
    
    return torch.tensor(X), torch.tensor(y)

# Preparando dados
X_train, y_train = preparar_dados(textos_pretreinamento, palavra_para_id)

print(f"📊 Dados preparados:")
print(f"Formato X: {X_train.shape} (batch_size, seq_length)")
print(f"Formato y: {y_train.shape} (batch_size,)")
print(f"\n🔍 Exemplo de treino:")
print(f"Input: {[id_para_palavra[id.item()] for id in X_train[0]]} -> Target: {id_para_palavra[y_train[0].item()]}")

In [ ]:
# Agora vamos treinar nosso modelo! 🏋️‍♂️
def treinar_modelo(modelo, X, y, epochs=100):
    losses = []
    modelo.train()
    
    for epoch in range(epochs):
        # Forward pass
        outputs = modelo(X)  # [batch_size, seq_len, vocab_size]
        # Pegamos só a última posição da sequência
        outputs = outputs[:, -1, :]  # [batch_size, vocab_size]
        
        # Calculando perda
        loss = criterion(outputs, y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")
    
    return losses

print("🔥 Iniciando treinamento...")
losses = treinar_modelo(modelo, X_train, y_train)
print("\n✅ Treinamento concluído! Liiindo!")

In [ ]:
# Visualizando o progresso do treinamento
plt.figure(figsize=(10, 6))
plt.plot(losses, linewidth=2, color='#e74c3c')
plt.title('📉 Evolução da Loss Durante o Treinamento', fontsize=16, fontweight='bold')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss (Cross Entropy)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"📊 Loss inicial: {losses[0]:.4f}")
print(f"📊 Loss final: {losses[-1]:.4f}")
print(f"📊 Melhoria: {((losses[0] - losses[-1]) / losses[0] * 100):.1f}%")

## 🎭 Fine-tuning: Especializando o Modelo

Agora que nosso modelo "aprendeu português básico", vamos especializá-lo!

É como um médico: primeiro ele estuda medicina geral (pré-treinamento), depois se especializa em cardiologia, pediatria, etc. (fine-tuning).

### Tipos de Fine-tuning:

1. **Supervised Fine-tuning (SFT)**: Com exemplos rotulados
2. **Instruction Tuning**: Ensinando a seguir instruções
3. **RLHF (Reinforcement Learning from Human Feedback)**: Aprendendo com feedback humano

**Dica do Pedro**: Fine-tuning é muito mais barato que pré-treinamento! É como customizar um carro em vez de construir do zero! 🚗

In [ ]:
# Vamos simular um fine-tuning para classificação de sentimentos
# Dados para fine-tuning (classificação de sentimentos)
dados_sentimento = [
    ("Que dia lindo hoje", "positivo"),
    ("Estou muito feliz", "positivo"), 
    ("Adoro esse lugar", "positivo"),
    ("Que dia terrível", "negativo"),
    ("Estou muito triste", "negativo"),
    ("Detesto essa situação", "negativo"),
    ("O tempo está normal", "neutro"),
    ("É apenas um dia comum", "neutro")
]

# Mapeamento de sentimentos
sentimento_para_id = {"positivo": 0, "negativo": 1, "neutro": 2}
id_para_sentimento = {0: "positivo", 1: "negativo", 2: "neutro"}

print("🎭 Dados para Fine-tuning (Análise de Sentimentos):")
for i, (texto, sentimento) in enumerate(dados_sentimento[:3]):
    print(f"  {i+1}. '{texto}' -> {sentimento}")
print("  ...")

print(f"\n📊 Classes: {list(sentimento_para_id.keys())}")

In [ ]:
# Modelo para classificação (usando o pré-treinado como base)
class ModeloClassificacao(nn.Module):
    def __init__(self, modelo_pretreinado, num_classes=3):
        super().__init__()
        # Usando as camadas do modelo pré-treinado
        self.embedding = modelo_pretreinado.embedding
        self.lstm = modelo_pretreinado.lstm
        
        # Nova cabeça para classificação
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),  # 128 é o hidden_dim do LSTM
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )
        
        # Congelando parâmetros do modelo base (opcional)
        # for param in self.embedding.parameters():
        #     param.requires_grad = False
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, _) = self.lstm(embedded)
        # Usando o último estado hidden para classificação
        output = self.classifier(hidden[-1])  # [batch_size, num_classes]
        return output

# Criando modelo de classificação baseado no pré-treinado
modelo_classificacao = ModeloClassificacao(modelo)
criterion_class = nn.CrossEntropyLoss()
optimizer_class = optim.Adam(modelo_classificacao.parameters(), lr=0.001)

print("🎯 Modelo de classificação criado!")
print(f"Parâmetros totais: {sum(p.numel() for p in modelo_classificacao.parameters())}")
print(f"Parâmetros treináveis: {sum(p.numel() for p in modelo_classificacao.parameters() if p.requires_grad)}")

## 📊 Comparando Estratégias de Treinamento

Existem várias formas de treinar um modelo. Vamos ver as principais:

### 1. **Training from Scratch** (Do Zero)
- ✅ Controle total
- ❌ Muito caro e demorado

### 2. **Fine-tuning Completo**
- ✅ Melhor performance
- ❌ Pode "esquecer" conhecimento anterior (catastrophic forgetting)

### 3. **Feature Extraction** (Congelando camadas)
- ✅ Rápido e barato
- ❌ Menos flexível

### 4. **LoRA (Low-Rank Adaptation)**
- ✅ Eficiente em memória
- ✅ Mantém conhecimento original
- ❌ Mais complexo de implementar

In [ ]:
# Visualizando as diferentes estratégias de treinamento
import matplotlib.patches as mpatches

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('🎯 Estratégias de Treinamento de LLMs', fontsize=16, fontweight='bold')

# Dados simulados para comparação
estrategias = ['From Scratch', 'Fine-tuning\nCompleto', 'Feature\nExtraction', 'LoRA']
custo = [100, 30, 5, 15]  # Custo relativo
tempo = [100, 25, 3, 12]  # Tempo relativo
performance = [95, 90, 75, 88]  # Performance relativa
memoria = [100, 100, 20, 45]  # Uso de memória relativo

# Gráfico de custo
axes[0,0].bar(estrategias, custo, color=['#e74c3c', '#f39c12', '#27ae60', '#3498db'])
axes[0,0].set_title('💰 Custo Relativo')
axes[0,0].set_ylabel('Custo (%)')

# Gráfico de tempo
axes[0,1].bar(estrategias, tempo, color=['#e74c3c', '#f39c12', '#27ae60', '#3498db'])
axes[0,1].set_title('⏱️ Tempo de Treinamento')
axes[0,1].set_ylabel('Tempo (%)')

# Gráfico de performance
axes[1,0].bar(estrategias, performance, color=['#e74c3c', '#f39c12', '#27ae60', '#3498db'])
axes[1,0].set_title('🎯 Performance')
axes[1,0].set_ylabel('Accuracy (%)')

# Gráfico de memória
axes[1,1].bar(estrategias, memoria, color=['#e74c3c', '#f39c12', '#27ae60', '#3498db'])
axes[1,1].set_title('🧠 Uso de Memória')
axes[1,1].set_ylabel('Memória (%)')

plt.tight_layout()
plt.show()

print("📊 Comparação das estratégias:")
print("• From Scratch: Máxima performance, máximo custo")
print("• Fine-tuning: Bom equilíbrio geral")
print("• Feature Extraction: Mais barato, menor performance")
print("• LoRA: Meio termo eficiente")

## 🏗️ Arquitetura do Processo de Treinamento

Vamos visualizar como funciona todo o pipeline de treinamento de um LLM:

```mermaid
graph TD
    A[📚 Coleta de Dados] --> B[🔤 Tokenização]
    B --> C[🎯 Pré-treinamento]
    C --> D[💾 Modelo Base]
    D --> E[🎭 Fine-tuning]
    E --> F[🧪 Avaliação]
    F --> G{✅ Performance OK?}
    G -->|Não| H[⚙️ Ajustar Hiperparâmetros]
    H --> E
    G -->|Sim| I[🚀 Modelo Final]
    
    style A fill:#3498db
    style D fill:#e74c3c
    style I fill:#27ae60
```

**Dica do Pedro**: Esse processo pode levar meses! O GPT-4 provavelmente levou mais de 6 meses para ficar pronto. Paciência é fundamental! ⏰

In [ ]:
# Vamos simular métricas de diferentes fases do treinamento
import pandas as pd

# Simulando dados de treinamento ao longo do tempo
fases = ['Semana 1', 'Semana 2', 'Semana 4', 'Semana 8', 'Semana 12', 'Semana 16']
pre_training_loss = [4.2, 3.8, 3.2, 2.9, 2.7, 2.6]
fine_tuning_acc = [None, None, None, None, 0.65, 0.82]
validation_loss = [4.5, 4.1, 3.4, 3.0, 2.8, 2.7]

# Criando DataFrame
df_metrics = pd.DataFrame({
    'Fase': fases,
    'Pre-training Loss': pre_training_loss,
    'Fine-tuning Accuracy': fine_tuning_acc,
    'Validation Loss': validation_loss
})

# Visualização das métricas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de Loss
ax1.plot(fases, pre_training_loss, marker='o', linewidth=3, label='Pre-training Loss', color='#e74c3c')
ax1.plot(fases, validation_loss, marker='s', linewidth=3, label='Validation Loss', color='#f39c12')
ax1.set_title('📉 Evolução da Loss', fontsize=14, fontweight='bold')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

# Gráfico de Accuracy (apenas fine-tuning)
fine_tune_fases = ['Semana 12', 'Semana 16']
fine_tune_acc = [0.65, 0.82]
ax2.plot(fine_tune_fases, fine_tune_acc, marker='D', linewidth=3, color='#27ae60', markersize=8)
ax2.set_title('🎯 Accuracy do Fine-tuning', fontsize=14, fontweight='bold')
ax2.set_ylabel('Accuracy')
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Observações importantes:")
print("• Loss de pré-treinamento diminui gradualmente")
print("• Fine-tuning começa após pré-treinamento")
print("• Validation loss acompanha training loss (sem overfitting)")
print("• Accuracy melhora rapidamente no fine-tuning")

## ⚡ Técnicas Avançadas: LoRA e QLoRA

Tá, mas e se eu quiser fazer fine-tuning de um modelo gigante como o Llama-2 70B? Meu computador vai explodir! 💥

É aí que entram as técnicas modernas:

### **LoRA (Low-Rank Adaptation)**
- Em vez de atualizar todos os pesos, criamos matrizes pequenas
- Reduz parâmetros treináveis em 90%+
- Mantém performance similar

### **QLoRA (Quantized LoRA)**
- LoRA + Quantização (4-bit)
- Permite treinar modelos 65B numa GPU 24GB!
- Revolução democrática do fine-tuning

**Analogia do Pedro**: É como reformar sua casa. Em vez de demolir tudo (fine-tuning completo), você só troca alguns móveis (LoRA). Muito mais barato e rápido! 🏠

In [ ]:
# Simulação simples de LoRA
class LoRALayer(nn.Module):
    def __init__(self, original_layer, rank=8):
        super().__init__()
        self.original_layer = original_layer
        
        # Congelando a camada original
        for param in self.original_layer.parameters():
            param.requires_grad = False
        
        # Criando matrizes LoRA pequenas
        in_features = original_layer.in_features
        out_features = original_layer.out_features
        
        self.lora_A = nn.Linear(in_features, rank, bias=False)
        self.lora_B = nn.Linear(rank, out_features, bias=False)
        self.scaling = 0.1  # Fator de escala
        
        # Inicialização
        nn.init.kaiming_uniform_(self.lora_A.weight)
        nn.init.zeros_(self.lora_B.weight)
    
    def forward(self, x):
        # Saída original + adaptação LoRA
        original_out = self.original_layer(x)
        lora_out = self.lora_B(self.lora_A(x)) * self.scaling
        return original_out + lora_out

# Exemplo de uso
camada_original = nn.Linear(512, 256)
camada_lora = LoRALayer(camada_original, rank=16)

# Comparando número de parâmetros
params_original = sum(p.numel() for p in camada_original.parameters())
params_lora = sum(p.numel() for p in camada_lora.parameters() if p.requires_grad)
params_total = sum(p.numel() for p in camada_lora.parameters())

print("🔍 Comparação LoRA vs Normal:")
print(f"Parâmetros originais: {params_original:,}")
print(f"Parâmetros LoRA (treináveis): {params_lora:,}")
print(f"Parâmetros totais: {params_total:,}")
print(f"Redução: {(1 - params_lora/params_original)*100:.1f}%")
print(f"\n💡 LoRA permite treinar com {params_lora/params_original*100:.1f}% dos parâmetros!")

In [ ]:
# Visualizando a eficiência do LoRA
ranks = [1, 2, 4, 8, 16, 32, 64]
reducao_params = []
memoria_estimada = []

in_features, out_features = 4096, 4096  # Tamanho típico de um LLM
params_full = in_features * out_features

for rank in ranks:
    params_lora = (in_features * rank) + (rank * out_features)
    reducao = (1 - params_lora/params_full) * 100
    reducao_params.append(reducao)
    
    # Estimativa de memória (assumindo float16)
    memoria_mb = (params_lora * 2) / (1024 * 1024)  # 2 bytes por parâmetro
    memoria_estimada.append(memoria_mb)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de redução de parâmetros
ax1.plot(ranks, reducao_params, marker='o', linewidth=3, color='#e74c3c', markersize=8)
ax1.set_title('📊 Redução de Parâmetros com LoRA', fontsize=14, fontweight='bold')
ax1.set_xlabel('Rank')
ax1.set_ylabel('Redução (%)')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(95, 100)

# Gráfico de uso de memória
ax2.plot(ranks, memoria_estimada, marker='s', linewidth=3, color='#3498db', markersize=8)
ax2.set_title('🧠 Uso de Memória (Parâmetros LoRA)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Rank')
ax2.set_ylabel('Memória (MB)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"🎯 Para uma camada 4096x4096:")
print(f"• Parâmetros completos: {params_full:,}")
print(f"• Com rank=16: {(in_features * 16) + (16 * out_features):,} parâmetros")
print(f"• Redução: {reducao_params[4]:.2f}%")
print(f"• Memória LoRA: {memoria_estimada[4]:.1f} MB")

## 🎯 Exercício Prático 1: Implementando um Mini Fine-tuning

Agora é sua vez! Vamos implementar um fine-tuning simples para classificação de texto.

**Seu desafio**:
1. Complete a função de preparação dos dados
2. Implemente o loop de treinamento
3. Calcule a acurácia no final

**Dica do Pedro**: Use os exemplos anteriores como base. Você consegue! 💪

In [ ]:
# 🏋️‍♂️ EXERCÍCIO 1: Complete o código abaixo

# Dados para o exercício
dados_exercicio = [
    ("Amo programar em Python", "positivo"),
    ("Machine Learning é fascinante", "positivo"),
    ("Adoro aprender coisas novas", "positivo"),
    ("Odeio bugs no código", "negativo"),
    ("Esse erro é muito chato", "negativo"),
    ("Estou frustrado com isso", "negativo"),
    ("O código funciona normal", "neutro"),
    ("É apenas uma função comum", "neutro")
]

def preparar_dados_exercicio(dados, palavra_para_id, sentimento_para_id):
    X, y = [], []
    
    for texto, sentimento in dados:
        # TODO: Tokenizar o texto e converter para IDs
        palavras = texto.lower().split()
        
        # Filtrar palavras que não estão no vocabulário
        ids = []
        for palavra in palavras:
            if palavra in palavra_para_id:
                ids.append(palavra_para_id[palavra])
        
        if len(ids) > 0:  # Só adicionar se tiver palavras válidas
            # TODO: Fazer padding ou truncar para tamanho fixo
            max_len = 5
            if len(ids) < max_len:
                ids.extend([0] * (max_len - len(ids)))  # Padding com 0
            else:
                ids = ids[:max_len]  # Truncar
            
            X.append(ids)
            y.append(sentimento_para_id[sentimento])
    
    return torch.tensor(X), torch.tensor(y)

# Preparar dados
X_exercise, y_exercise = preparar_dados_exercicio(dados_exercicio, palavra_para_id, sentimento_para_id)

print(f"✅ Dados preparados para exercício:")
print(f"Shape X: {X_exercise.shape}")
print(f"Shape y: {y_exercise.shape}")
print(f"Exemplo: {X_exercise[0]} -> {id_para_sentimento[y_exercise[0].item()]}")

# TODO: Agora complete o treinamento!
# Dica: Use o modelo_classificacao que criamos anteriormente

## 🚨 Desafios do Treinamento de LLMs

Nem tudo são flores no mundo dos LLMs! Existem vários desafios:

### 🔥 **Principais Problemas:**

1. **Catastrophic Forgetting**: Modelo "esquece" conhecimento anterior
2. **Data Contamination**: Dados de teste "vazam" no treinamento
3. **Bias e Toxicidade**: Modelo replica preconceitos dos dados
4. **Hallucination**: Modelo "inventa" informações
5. **Custo Computacional**: Treinar é caríssimo!

### 💡 **Soluções:**
- **Regularização**: Para evitar overfitting
- **Curriculum Learning**: Treinar do fácil para o difícil
- **Data Filtering**: Limpar dados de treinamento
- **Constitutional AI**: Ensinar princípios éticos

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-07_img_02.png)

In [ ]:
# Simulando o problema de Catastrophic Forgetting
import numpy as np

# Simulação de performance em diferentes tarefas
tarefas = ['Tarefa A\n(Original)', 'Tarefa B\n(Fine-tuning)', 'Tarefa A\n(Após B)', 'Tarefa C\n(Fine-tuning)', 'Tarefa A\n(Após C)']
performance_sem_estrategia = [85, 90, 60, 88, 45]  # Com catastrophic forgetting
performance_com_estrategia = [85, 88, 82, 86, 80]  # Com regularização

x = np.arange(len(tarefas))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, performance_sem_estrategia, width, label='Sem Estratégia', color='#e74c3c', alpha=0.8)
bars2 = ax.bar(x + width/2, performance_com_estrategia, width, label='Com Regularização', color='#27ae60', alpha=0.8)

ax.set_title('🧠 Catastrophic Forgetting em Ação', fontsize=16, fontweight='bold')
ax.set_ylabel('Performance (%)')
ax.set_xlabel('Sequência de Treinamento')
ax.set_xticks(x)
ax.set_xticklabels(tarefas)
ax.legend()
ax.grid(True, alpha=0.3)

# Adicionando valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}%',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3),
                   textcoords="offset points",
                   ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("🔍 Observações:")
print("• Performance na Tarefa A cai drasticamente sem estratégias")
print("• Regularização ajuda a manter conhecimento anterior")
print("• É um trade-off entre aprender novo e manter antigo")
print("\n💡 Soluções: EWC, L2 regularization, rehearsal learning")

## 🔮 Pipeline Completo: Do Texto Bruto ao Modelo

Vamos ver todo o processo completo, do começo ao fim:

```mermaid
graph TB
    subgraph "📚 Dados"
        A[Web Scraping] --> B[Livros]
        C[Wikipedia] --> B
        D[Artigos] --> B
        B --> E[Dataset Bruto]
    end
    
    subgraph "🧹 Pré-processamento"
        E --> F[Limpeza]
        F --> G[Deduplicação]
        G --> H[Tokenização]
    end
    
    subgraph "🏋️ Treinamento"
        H --> I[Pré-treinamento]
        I --> J[Modelo Base]
        J --> K[Fine-tuning]
        K --> L[RLHF]
    end
    
    subgraph "🎯 Resultado"
        L --> M[Modelo Final]
    end
    
    style E fill:#3498db
    style J fill:#f39c12
    style M fill:#27ae60
```

In [ ]:
# Estimativas de recursos para diferentes tamanhos de modelo
modelos = ['GPT-2\n(1.5B)', 'GPT-3\n(175B)', 'PaLM\n(540B)', 'GPT-4\n(~1T)']
parametros = [1.5, 175, 540, 1000]  # Em bilhões
custo_treinamento = [50000, 4600000, 15000000, 100000000]  # Em dólares (estimativa)
gpus_necessarias = [8, 1024, 3000, 10000]  # GPUs A100
tempo_treinamento = [7, 30, 45, 90]  # Em dias

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('💰 Recursos Necessários por Tamanho de Modelo', fontsize=16, fontweight='bold')

colors = ['#3498db', '#e74c3c', '#f39c12', '#9b59b6']

# Parâmetros
bars1 = ax1.bar(modelos, parametros, color=colors)
ax1.set_title('📊 Número de Parâmetros')
ax1.set_ylabel('Bilhões de Parâmetros')
ax1.set_yscale('log')

# Custo
bars2 = ax2.bar(modelos, custo_treinamento, color=colors)
ax2.set_title('💸 Custo de Treinamento')
ax2.set_ylabel('Dólares (USD)')
ax2.set_yscale('log')

# GPUs
bars3 = ax3.bar(modelos, gpus_necessarias, color=colors)
ax3.set_title('🖥️ GPUs Necessárias')
ax3.set_ylabel('Número de GPUs A100')
ax3.set_yscale('log')

# Tempo
bars4 = ax4.bar(modelos, tempo_treinamento, color=colors)
ax4.set_title('⏰ Tempo de Treinamento')
ax4.set_ylabel('Dias')

# Adicionando valores nas barras
for ax, values in [(ax1, parametros), (ax2, custo_treinamento), (ax3, gpus_necessarias), (ax4, tempo_treinamento)]:
    bars = ax.patches
    for bar, value in zip(bars, values):
        height = bar.get_height()
        if ax == ax2:  # Custo em milhões
            label = f'${value/1000000:.1f}M' if value >= 1000000 else f'${value/1000:.0f}K'
        elif ax == ax1:  # Parâmetros
            label = f'{value}B'
        else:
            label = f'{value}'
        
        ax.annotate(label,
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3),
                   textcoords="offset points",
                   ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print("🤯 Fatos impressionantes:")
print(f"• GPT-4 custou ~$100M para treinar (estimativa)")
print(f"• Precisou de ~10,000 GPUs A100 por 3 meses")
print(f"• Isso é mais que o PIB de alguns países pequenos!")
print(f"• Por isso que fine-tuning é tão importante! 💡")

## 🎯 Exercício Final: Análise de Estratégias

**Cenário**: Você trabalha numa startup e precisa criar um chatbot para atendimento ao cliente. Você tem:
- Orçamento: $10,000
- Prazo: 2 semanas 
- Hardware: 1 GPU A100
- Dataset: 50,000 conversas de atendimento

**Sua missão**: Escolha a melhor estratégia e justifique!

**Opções**:
1. Treinar do zero um modelo pequeno
2. Fine-tuning completo do Llama-2 7B
3. LoRA fine-tuning do Llama-2 13B
4. Prompt engineering com GPT-4 via API

In [ ]:
# 🤔 EXERCÍCIO FINAL: Análise de estratégias
# Complete a análise abaixo!

estrategias_analise = {
    'Treinar do Zero': {
        'custo': 8000,
        'tempo_dias': 10,
        'performance_estimada': 70,
        'controle': 100,
        'risco': 80
    },
    'Fine-tuning Llama-2 7B': {
        'custo': 2000,
        'tempo_dias': 3,
        'performance_estimada': 85,
        'controle': 80,
        'risco': 30
    },
    'LoRA Llama-2 13B': {
        'custo': 1500,
        'tempo_dias': 2,
        'performance_estimada': 88,
        'controle': 70,
        'risco': 20
    },
    'GPT-4 API': {
        'custo': 5000,  # Custo mensal estimado
        'tempo_dias': 1,
        'performance_estimada': 92,
        'controle': 30,
        'risco': 40  # Dependência externa
    }
}

# Visualização da análise
import pandas as pd

df_estrategias = pd.DataFrame(estrategias_analise).T
print("📊 Análise das Estratégias:")
print(df_estrategias)

# TODO: Complete sua análise aqui!
print("\n🤔 Minha escolha seria: ___________")
print("\n✅ Justificativa:")
print("1. ___________")
print("2. ___________")
print("3. ___________")

# Dica do Pedro: Considere o trade-off entre custo, tempo, performance e controle!

In [ ]:
# Visualização final da análise de estratégias
fig, ax = plt.subplots(figsize=(12, 8))

# Criando scatter plot
for i, (nome, dados) in enumerate(estrategias_analise.items()):
    # Plotando custo vs performance, com tamanho do ponto = inverso do tempo
    size = 1000 / dados['tempo_dias']  # Quanto menor o tempo, maior o ponto
    ax.scatter(dados['custo'], dados['performance_estimada'], 
              s=size, alpha=0.7, label=nome)
    
    # Adicionando rótulos
    ax.annotate(nome, 
               (dados['custo'], dados['performance_estimada']),
               xytext=(10, 10), textcoords='offset points',
               fontsize=10, fontweight='bold')

ax.set_xlabel('💰 Custo (USD)', fontsize=12)
ax.set_ylabel('🎯 Performance Estimada (%)', fontsize=12)
ax.set_title('⚖️ Trade-off: Custo vs Performance\n(Tamanho do ponto = Velocidade)', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# Adicionando linhas de referência
ax.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='Performance Mínima')
ax.axvline(x=10000, color='orange', linestyle='--', alpha=0.5, label='Orçamento Máximo')

plt.tight_layout()
plt.show()

print("🎯 Qual estratégia você escolheria? Pense em:")
print("• Restrições de orçamento e tempo")
print("• Performance necessária")
print("• Controle sobre o modelo")
print("• Riscos envolvidos")

## 🎉 Resumo do Módulo: O que Aprendemos?

Ufa! Que jornada, né pessoal? Vamos recapitular o que vimos neste módulo:

### 🏆 **Conceitos Principais:**
1. **Pré-treinamento**: Como LLMs aprendem linguagem (auto-supervisionado)
2. **Fine-tuning**: Especializando modelos para tarefas específicas
3. **LoRA/QLoRA**: Técnicas eficientes para fine-tuning
4. **Desafios**: Catastrophic forgetting, custo, bias

### 🛠️ **Técnicas Aprendidas:**
- Preparação de dados para treinamento
- Implementação de loops de treinamento
- Estratégias de fine-tuning
- Análise de trade-offs

### 🔗 **Conexões com Módulos Anteriores:**
- **Tokens** (Módulo 4): Base para preparar dados de treinamento
- **Embeddings** (Módulo 5): Como o modelo aprende representações
- **Arquitetura** (Módulo 3): Como os transformers processam durante treinamento

### 🚀 **Preparando para o Próximo Módulo:**
No **Módulo 8 (Prompting e Engenharia)**, vamos ver como usar esses modelos treinados de forma eficiente, sem precisar retreiná-los!

**Dica Final do Pedro**: Treinamento é como cozinhar - você pode fazer do zero (caro e demorado) ou usar ingredientes prontos e temperar do seu jeito (fine-tuning)! 👨‍🍳

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/introdução-à-llms-modulo-07_img_03.png)

In [ ]:
# 🎓 Certificado de conclusão do módulo!
import datetime

print("="*60)
print("🎓 CERTIFICADO DE CONCLUSÃO 🎓")
print("="*60)
print(f"""   
    Parabéns! Você concluiu o Módulo 7:
    
    📚 TREINAMENTO E PRÉ-TREINAMENTO DE LLMs
    
    Conceitos dominados:
    ✅ Pré-treinamento auto-supervisionado
    ✅ Fine-tuning e especialização
    ✅ Técnicas LoRA e QLoRA
    ✅ Desafios e soluções do treinamento
    ✅ Análise de trade-offs e estratégias
    
    Data: {datetime.datetime.now().strftime('%d/%m/%Y')}
    Instrutor: Pedro Nunes Guth
    
    🚀 Próximo: Módulo 8 - Prompting e Engenharia
""")
print("="*60)
print("\n🔥 Você está no caminho certo para dominar LLMs!")
print("💪 Continue praticando e experimentando!")
print("📚 Nos vemos no próximo módulo!")

# Easter egg: Estatísticas do módulo
print("\n📊 Estatísticas deste módulo:")
print(f"• Células de código executadas: {len([cell for cell in globals() if not cell.startswith('_')])}")
print(f"• Conceitos apresentados: 15+")
print(f"• Gráficos criados: 8")
print(f"• Exercícios propostos: 2")
print(f"• Dicas do Pedro: 10+")
print("\n🎯 Você está pronto para os prompts! Bora pro Módulo 8! 🚀")